# 开始前的准备

CoreFWRouteRulesOutput.ipynb - 核心交换机的NAT规则导出脚本

1. 创建venv

`pyton -m venv venv`

2. 安装jupyter

`pip install jupyter pandas openpyxl`

## 代码开始

In [ ]:
# 请求地址
# https://10.8.8.253:8443/common.html
# 读取原始的 payload
with open(file="./data/raw-02.payload", mode="r", encoding="UTF-8") as f:
    raw_payload = f.read()

print(raw_payload)

# 看起来像是 JSON, 先处理下
step1_payload: str = raw_payload.replace("&quot;", '"')
print(step1_payload)

# 读取并且格式化 json
import json
step2_payload: dict = json.loads(step1_payload)
with open(file="./data/step2_payload.json", mode="w", encoding="UTF-8") as f:
    # ensure_ascii: 不要将非ASCII字符转换成Unicode转义序列, 而是直接输出这些字符
    f.write(json.dumps(step2_payload, ensure_ascii=False))

# 多层嵌套的字典, 需要读取到 data 层
step3_payload: list = step2_payload["getAddressMapList"]["data"]
print(type(step3_payload))
with open(file="./data/step3_payload.json", mode="w", encoding="UTF-8") as f:
    # ensure_ascii: 不要将非ASCII字符转换成Unicode转义序列, 而是直接输出这些字符
    f.write(json.dumps(step3_payload, ensure_ascii=False))

# 后续对 step3_payload list按索引取值, 即可获取任一一个配置的详细配置 json
# 以其中的一个子配置为例
#     "name": "tcp-BIM模型-80",     映射名称
#     "id": 119,                    映射配置ID
#     "outerAddr": [                公网地址
#         "58.49.78.142"
#     ],
#     "innerAddr": [
#         "172.28.13.205"           私网地址
#     ],
#     "reverse": "false",
#     "portPAT": "true",
#     "zone": "",
#     "protocol": "tcp",            指定协议
#     "outerPort": "8080",          公网端口
#     "innerPort": "80",            私网端口
#     "state": "no-check",
#     "route": false,
#     "enable": true                启用配置

# 它说, 可以上了 ———— ywuwuyi

import pandas as pd

for i in step3_payload:
    
    # 清理无用的 key
    del i['reverse'] 
    del i['portPAT']
    del i['zone']
    del i['state']
    del i['route']
    # 将 list 中的字符串直接取出来
    i['outerAddr'] = i['outerAddr'][0]
    i['innerAddr'] = i['innerAddr'][0]

    # 读取对象并且将 dict 全部的 value 转换成字符串
    df = pd.DataFrame([i])
    df = df.astype(str)

    # 指定Excel文件路径
    excel_path = "./data/step4_payload.xlsx"

    # 检查文件是否存在，如果存在则读取，否则创建一个新的DataFrame
    try:
        # 尝试读取Excel文件
        book = pd.read_excel(excel_path)
        # 将新的DataFrame追加到Excel文件中
        book = pd.concat([book, df], ignore_index=True)
    except FileNotFoundError:
        # 文件不存在，创建一个新的DataFrame
        book = df

    # 将DataFrame写入Excel文件
    book.to_excel(excel_path, index=False)

    print("数据已成功追加到Excel文件。")

{&quot;getAddressMapList&quot;:{&quot;startLine&quot;:101,&quot;totalCount&quot;:121,&quot;data&quot;:[{&quot;name&quot;:&quot;tcp-BIM模型-80&quot;,&quot;id&quot;:119,&quot;outerAddr&quot;:[&quot;58.49.78.142&quot;],&quot;innerAddr&quot;:[&quot;172.28.13.205&quot;],&quot;reverse&quot;:&quot;false&quot;,&quot;portPAT&quot;:&quot;true&quot;,&quot;zone&quot;:&quot;&quot;,&quot;protocol&quot;:&quot;tcp&quot;,&quot;outerPort&quot;:&quot;8080&quot;,&quot;innerPort&quot;:&quot;80&quot;,&quot;state&quot;:&quot;no-check&quot;,&quot;route&quot;:false,&quot;enable&quot;:true},{&quot;name&quot;:&quot;tcp-BIM模型-808&quot;,&quot;id&quot;:120,&quot;outerAddr&quot;:[&quot;58.49.78.142&quot;],&quot;innerAddr&quot;:[&quot;172.28.13.205&quot;],&quot;reverse&quot;:&quot;false&quot;,&quot;portPAT&quot;:&quot;true&quot;,&quot;zone&quot;:&quot;&quot;,&quot;protocol&quot;:&quot;tcp&quot;,&quot;outerPort&quot;:&quot;808&quot;,&quot;innerPort&quot;:&quot;808&quot;,&quot;state&quot;:&quot;no-check&quot;,&quot;route